In [1]:
import re
from pathlib import Path
from os.path import abspath
import os
import numpy as np
import pandas as pd
import random
import pickle
import csv
import subprocess

In [2]:
print(os.getcwd())

/rdma/vast-rdma/vast/cpl/ckauf/perturbed-neural-nlp/analysis/pmi_verification


In [3]:
stimuli_path = abspath('../../ressources/scrambled_stimuli_dfs')
print(stimuli_path)

/rdma/vast-rdma/vast/cpl/ckauf/perturbed-neural-nlp/analysis/pmi_verification/stimuli_csvs


In [5]:
fname = "stimuli_Original.pkl"
with open(os.path.join(stimuli_path,fname), 'rb') as f:
    df = pickle.load(f)
df

,0,1,2,3
0,243sentences.0,Original,beekeeping encourages the conservation of loca...,1.084573
1,243sentences.1,Original,it is in every beekeeper's interest to conserv...,1.196020
2,243sentences.2,Original,as a passive form of agriculture it does not r...,1.417875
3,243sentences.3,Original,beekeepers also discourage the use of pesticid...,1.256986
4,243sentences.4,Original,artisanal beekeepers go to extremes for their ...,1.073480
...,...,...,...,...
622,384sentences.379,Original,some windows have multiple panes to increase i...,1.195439
623,384sentences.380,Original,a woman is a female human adult.,1.100778
624,384sentences.381,Original,a woman is stereotypically seen as a caregiver.,1.387173
625,384sentences.382,Original,a woman can become pregnant and bear children.,1.362809


In [80]:
# Mollica calculate PMI input style is:
# * 1. column is stimulus_id
# * 2. column is condition
# * 3. column is sentence

In [81]:
conditions = ['Original', 'Scr1', 'Scr3', 'Scr5', 'Scr7', 'lowPMI', 'lowPMI_random', 'backward', 'random', 'random_poscontrolled', 'random_withreplacement']

In [82]:
frames = []

savedir = abspath('./stimuli_csvs')
os.makedirs(savedir, exist_ok=True)

for cond in conditions:
    for filename in os.listdir(stimuli_path):
        if not ".pkl" in filename:
            continue
        if filename.split(".")[0].split("stimuli_")[1] == cond:
            print(cond)
            with open(os.path.join(stimuli_path,filename), 'rb') as f:
                df = pickle.load(f)
            ids = list(df["stimulus_id"])
            conds = list([cond]*len(ids))
            sentences = list(df["sentence"])
            #print(ids[:5], conds[:5], sentences[:5])
            #print("*"*30)
            curr_df = pd.DataFrame({
                "id" : ids,
                "condition" : conds,
                "sentence" : sentences
            })
            
    curr_df.to_csv(f"{savedir}/stimuli_{cond}.csv", header=None, index=False)
    frames.append(curr_df)

Original
Scr1
Scr3
Scr5
Scr7
lowPMI
lowPMI_random
backward
random
random_poscontrolled
random_withreplacement


In [83]:
full_df = pd.concat(frames)
full_df

,id,condition,sentence
0,243sentences.0,Original,beekeeping encourages the conservation of loca...
1,243sentences.1,Original,it is in every beekeeper's interest to conserv...
2,243sentences.2,Original,as a passive form of agriculture it does not r...
3,243sentences.3,Original,beekeepers also discourage the use of pesticid...
4,243sentences.4,Original,artisanal beekeepers go to extremes for their ...
...,...,...,...
622,384sentences.379,random_withreplacement,and the depending allow watching throwing ofte...
623,384sentences.380,random_withreplacement,used things that connected and a to.
624,384sentences.381,random_withreplacement,joined the the drunk the or have due.
625,384sentences.382,random_withreplacement,controlled barrier butterfat on articles their...


In [84]:
print(list(full_df.loc[full_df["condition"] == "random_poscontrolled"]["sentence"])[:5])

['part go to silk and are impressive.', 'or inside air have bear spent is a internet 1970s gambling walls subject of.', 'cutting to predate which but india to even south presence use with hit a water cod lakes woodwind cooked made.', 'flood shots types be a the ice rain and have an or of quietly activity.', "an small or in it compound 's live salmon trousers water the lot beak were."]


In [85]:
full_df.to_csv(f"{savedir}/all_scrambled_stimuli.csv", header=None, index=False)